In [ ]:
from soview import *
import shutil

### set SODB root path

In [ ]:
root_path = 'zhiyuanyuan/projects/SODB/'

### h5ad path

In [ ]:
source_h5ad_path = 'zhiyuanyuan/PUBDT/st/dbit-seq/data_h5ad'
source_h5ad_data = 'GSM4189615_0719cL_gene'
source_h5ad = f'{source_h5ad_path}/{source_h5ad_data}.h5ad'

### data info

In [ ]:
biotech_category = 'Spatial_MultiOmics'
biotech = 'DBiT-seq'
dataset_short = 'liu2020high'
data_name = source_h5ad_data


### mkdir

In [ ]:
import os

path_rel = f'demo_data/{biotech_category}/{biotech}/{dataset_short}/{data_name}'
path_abs = root_path+path_rel
os.makedirs(path_abs, exist_ok=False)
print("The new directory is created!")

### move h5ad

In [ ]:
shutil.copyfile(source_h5ad, f'{path_abs}/data.h5ad')

### load data

In [ ]:
data_use = path_rel
adata = ad.read_h5ad(f'{root_path}{data_use}/data.h5ad')

In [ ]:
print(f'h5ad path: {data_use}/data.h5ad')

In [ ]:
adata.layers["counts"] = adata.X.copy()

### basic processing

In [ ]:
adata

In [ ]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)



In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)

In [ ]:
sc.pl.embedding(adata,basis='spatial',color='leiden')

In [ ]:
adata

In [ ]:
sc.tl.umap(adata,n_components=3)

In [ ]:
SOViewer_plot(adata,embedding_use='X_umap',dot_size=10)
print(f'View path: {data_use}/View_UMAP_LAB.pdf')

### write features

In [ ]:
gene_list = np.array(adata.var_names).tolist()


In [ ]:
obs_list = np.array(adata.obs.keys()).tolist()

In [ ]:
obsm_list = []
for obsm in np.array(adata.obsm.keys()).tolist():
    for i in range(adata.obsm[obsm].shape[1]):
        cur_obsm = f'{obsm}@{i}'
        obsm_list.append(cur_obsm)

In [ ]:
feature_dict = {
    'gene':gene_list,
    'obs':obs_list,
    'obsm':obsm_list
}

In [ ]:
import pickle
with open(f'{root_path}{data_use}/feature_dict.pickle', 'wb') as handle:
    pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print(f'把feature_dict写为: {data_use}/feature_dict.pickle')

### dumo h5ad

In [ ]:
adata.write_h5ad(f'{root_path}{data_use}/data.h5ad')

### output

In [ ]:
print(source_h5ad_data)
print(f'h5ad path: {data_use}/data.h5ad')
print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
print(f'N_Unit: {adata.shape[0]}')
print(f'N_Gene: {adata.shape[1]}')